In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import Callable, List, Optional, Tuple
import time
RAND_SEED = 11

In [2]:
# dynamics of the pendulum
@dataclass
class Gaussian:
    mean: np.ndarray
    cov: np.ndarray


DT = 0.01
G = sp.constants.g


def xkp1(xk: np.ndarray) -> np.ndarray:
    return np.array([xk[0] + xk[1]*DT,
                     xk[1] - G*np.sin(xk[0])*DT])


def get_q() -> np.ndarray:
    QC = 0.1
    return np.array([[QC*DT**3/3, QC*DT**2/2],
                     [QC*DT**2/2, QC*DT]])


def yk(xk: np.ndarray) -> np.ndarray:
    return np.array([np.sin(xk[0])])

In [3]:
# setup the problem

@dataclass
class Observations:
    times: np.ndarray
    obs_ind: np.ndarray  # index of times that are observed
    obs: np.ndarray
    names: List[str]


def generate_truth(x0: np.ndarray, dt: float, N: int) -> Observations:
    times = np.zeros(N+1)  # from 0 to N times
    xk = np.zeros((N+1, 2))  # state at each time
    xk[0] = x0
    for i in range(1, N+1):
        times[i] = i*dt
        xk[i] = xkp1(xk[i-1])
    return Observations(times, np.arange(N+1), xk, [r'$x_1$ true', r'$x_2$ true'])


def generate_obs(truth: Observations, delta: float, R: float) -> Observations:
    process_std = np.linalg.cholesky(get_q())
    measure_std = np.sqrt(R)
    measure_ind = np.arange(delta, truth.obs_ind[-1]+1, delta)
    obs = np.zeros((measure_ind.size, 1))
    obs_ind = 0
    for i in measure_ind:
        obs[obs_ind] = yk(truth.obs[i] +
                          np.dot(process_std, np.random.randn(2))) + measure_std*np.random.randn()
        obs_ind += 1
    return Observations(truth.times, measure_ind, obs, [r'$x_1$ obs'])


@dataclass
class ObservationData:
    delta: float
    R: float
    obs: Observations


np.random.seed(RAND_SEED)
N = 500
X0 = np.array([1.5, 0])
prior = Gaussian(X0, np.eye(2))
process_cov = Gaussian(np.zeros(2), get_q())
truth = generate_truth(X0, DT, N)
deltas = [5, 10, 20, 40]
Rs = [0.001, 0.01, 0.1, 1]
data = []
for delta in deltas:
    for R in Rs:
        data.append(ObservationData(delta, R, generate_obs(truth, delta, R)))

ValueError: could not broadcast input array from shape (2,1) into shape (2,)

In [ ]:
# setup helper functions


@dataclass
class KFTracker:
    means: np.ndarray
    covs: np.ndarray
    stds: np.ndarray


def plot_data_and_truth(fignum, title, true_obs: Observations, data_obs: List[ObservationData],
                        kft: Optional[List[KFTracker]] = None) -> plt.figure:

    colors = ['red', 'blue']
    plt.close(fignum)
    fig, axs = plt.subplots(4, 4, num=fignum, figsize=(15, 15), sharex=True, sharey=True)
    fig.tight_layout(pad=3)
    fig.suptitle(title, fontsize=18, y=1.0)
    for i in range(len(data_obs)):
        ax = axs[i//4, i % 4]
        ax.plot(true_obs.times, true_obs.obs[:, 0], color=colors[0], label=true_obs.names[0])
        ax.plot(true_obs.times, true_obs.obs[:, 1], color=colors[1], label=true_obs.names[1])

        if kft is not None:
            ax.plot(data_obs[i].obs.times, kft[i].means[:, 0],
                    '--', color=colors[0], label=r'$x_1$ est')
            ax.plot(data_obs[i].obs.times, kft[i].means[:, 1],
                    '--', color=colors[1], label=r'$x_2$ est')
            ax.fill_between(data_obs[i].obs.times,
                            kft[i].means[:, 0] - 2 * kft[i].stds[:, 0],
                            kft[i].means[:, 0] + 2 * kft[i].stds[:, 0],
                            color=colors[0], alpha=0.3)
            ax.fill_between(data_obs[i].obs.times,
                            kft[i].means[:, 1] - 2 * kft[i].stds[:, 1],
                            kft[i].means[:, 1] + 2 * kft[i].stds[:, 1],
                            color=colors[1], alpha=0.3)
            ax.set_title(
                r'$\delta$=' +
                f'{data_obs[i].delta};R={data_obs[i].R};MSE={mean_sq_error(true_obs, kft[i]):.3f}',
                fontsize=12)
        else:
            ax.set_title(r'$\delta$='+f'{data_obs[i].delta};R={data_obs[i].R}', fontsize=12)

        ax.plot(data_obs[i].obs.times[data_obs[i].obs.obs_ind], data_obs[i].obs.obs,
                'o', color=colors[0], alpha=0.4, label=data_obs[i].obs.names[0])

        ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}'))
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0f}'))

    h, l = axs[0, 0].get_legend_handles_labels()
    fig.legend(h, l, loc='lower right', fontsize=12, ncol=3)
    fig.supxlabel('Time', fontsize=14)
    fig.supylabel('State Estimate', fontsize=14)
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    fig.savefig(f'figs/{title}.svg')
    plt.show()
    return fignum + 1


def mean_sq_error(truth: Observations, results: KFTracker) -> float:
    return np.mean((truth.obs - results.means)**2)


fignum = 1

fignum = plot_data_and_truth(fignum, 'Pendulum trajectories and data', truth, data)

In [ ]:
# extended kalman filter

def ekf_predict(phi: Callable[[np.ndarray], np.ndarray], A: np.ndarray,
                X: Gaussian, xi: Gaussian) -> Gaussian:
    """Linear Prediction Step: Propagate uncertainty for one time step.

    X_{k+1} = A X_{k} + xi, xi sim mathcal{N}{noise_mean, noise_cov}
    X_{k} sim mathcal{N}(mean, cov)

    Inputs
    ------
    phi: dynamics function
    A: (d, d), linearized dynamics
    X: Gaussian
    xi: Gaussian

    Returns
    -------
    A Gaussian random variable for the next time step
    """

    pred_mean = phi(X.mean) + xi.mean
    pred_cov = np.dot(A, np.dot(X.cov, A.T)) + xi.cov
    return Gaussian(pred_mean, pred_cov)


def ekf_update(data: np.ndarray, h: Callable[[np.ndarray], np.ndarray], H: np.ndarray,
               X: Gaussian, eta: Gaussian) -> Gaussian:
    """Linear Gaussian Inverse Problem.

    Inputs
    ------
    data: (N) array of data
    h: observation function
    H: (m, d), linearized observation model
    X: Gaussian
    eta: Gaussian

    Returns
    -------
    Gaussian Posterior
    """
    mu = h(X.mean) + eta.mean
    U = np.dot(X.cov, H.T)
    S = np.dot(H, U) + eta.cov

    update_mean = X.mean + np.dot(U, np.linalg.solve(S, data - mu))
    update_cov = X.cov - np.dot(U, np.linalg.solve(S, U.T))
    return Gaussian(update_mean, update_cov)


def extended_kalman_filter(data: Observations,
                           phi: Callable[[np.ndarray], np.ndarray],
                           A: Callable[[np.ndarray], np.ndarray],
                           h: Callable[[np.ndarray], np.ndarray],
                           H: Callable[[np.ndarray], np.ndarray],
                           prior: Gaussian,
                           xi: Gaussian, eta: Gaussian) -> Tuple[np.ndarray, np.ndarray]:
    """The Kalman filter.

    Inputs
    ------
    data: (N, m), N is the number of time steps, m is the size of the observations
    phi: dynamics function
    A: linearized dynamics function
    h: observation function
    H: linearized observation function
    prior: prior Gaussian
    xi: process noise
    eta: measurement noise

    Return
    ------
    KFTracker
    """

    num_steps = data.times.shape[0]

    d = prior.mean.shape[0]
    mean_store = np.zeros((num_steps, d))
    mean_store[0, :] = np.copy(prior.mean)
    cov_store = np.zeros((num_steps, d, d))
    cov_store[0, :, :] = np.copy(prior.cov)

    std_store = np.zeros((num_steps, d))
    std_store[0, :] = np.sqrt(np.diag(cov_store[0, :, :]))

    # Loop over all time steps
    Xnext = prior
    on_obs = 0
    for ii in range(1, num_steps):
        # Prediction
        Xpred = ekf_predict(phi, A(Xnext.mean), Xnext, xi)

        # We have an observation so an update must occur
        if on_obs < data.obs_ind.shape[0] and ii == data.obs_ind[on_obs]:
            y = data.obs[on_obs]
            on_obs += 1

            # Update
            Xup = ekf_update(y, h, H(Xpred.mean), Xpred, eta)
            Xnext = Xup

        else:
            Xnext = Xpred

        mean_store[ii, :] = np.copy(Xnext.mean)
        cov_store[ii, :, :] = np.copy(Xnext.cov)
        std_store[ii, :] = np.sqrt(np.diag(cov_store[ii, :, :]))

    return KFTracker(mean_store, cov_store, std_store)

# linearized dynamics


def Ak(xk: np.ndarray) -> np.ndarray:
    return np.array([[1, DT],
                     [-G*np.cos(xk[0])*DT, 1]])


def Hk(xk: np.ndarray) -> np.ndarray:
    return np.array([[np.cos(xk[0]), 0]]).reshape(1, 2)


np.random.seed(RAND_SEED)
ekf_results = []
ekf_start = time.time()
for obs in data:
    ekf_results.append(extended_kalman_filter(obs.obs, xkp1, Ak, yk,
                                              Hk, prior, process_cov, Gaussian(0, obs.R)))
print(f'EKF took {time.time() - ekf_start:.3f} seconds')

for result, obs in zip(ekf_results, data):
    mse = mean_sq_error(truth, result)
    print(f'delta={obs.delta} \tR={obs.R}\tMSE={mse:.1e}')

fignum = plot_data_and_truth(fignum, 'Pendulum trajectories with EKF', truth, data, ekf_results)

In [ ]:
# unscented kalman filter of order 3

def unscented_points(X: Gaussian, alg: str = 'chol', alpha: float = 1, beta: float = 0, kappa: float = 0):
    """Generate unscented points"""
    dim = X.cov.shape[0]
    lam = alpha*alpha*(dim + kappa) - dim
    if alg == "chol":
        L = np.linalg.cholesky(X.cov)
    elif alg == "svd":
        u, s, v = np.linalg.svd(X.cov)
        L = np.dot(u, np.sqrt(np.diag(s)))
    pts = np.zeros((2*dim+1, 2))
    pts[0, :] = X.mean
    for ii in range(1, dim+1):
        pts[ii, :] = X.mean + np.sqrt(dim + lam)*L[:, ii-1]
        pts[ii+dim, :] = X.mean - np.sqrt(dim + lam)*L[:, ii-1]

    W0m = lam / (dim + lam)
    W0C = lam / (dim + lam) + (1 - alpha*alpha + beta)
    Wim = 1/2 / (dim + lam)
    Wic = 1/2 / (dim + lam)
    return pts, (W0m, Wim, W0C, Wic)


def ukf_predict(phi: Callable[[np.ndarray], np.ndarray], X: Gaussian, xi: Gaussian) -> Gaussian:
    """Linear Prediction Step: Propagate uncertainty for one time step.

    X_{k+1} = A X_{k} + xi, xi sim mathcal{N}{noise_mean, noise_cov}
    X_{k} sim mathcal{N}(mean, cov)

    Inputs
    ------
    phi: dynamics function
    X: Gaussian
    xi: Gaussian

    Returns
    -------
    A Gaussian random variable for the next time step
    """
    d, (W0m, Wim, W0C, Wic) = unscented_points(X)
    w = 
    # pred_mean = phi(X.mean) + xi.mean
    pred_mean = phi(Xpts[0]) * W0m
    for ii in range(1, Xpts.shape[0]):
        pred_mean += phi(Xpts[ii]) * Wim

    # pred_cov = np.dot(A, np.dot(X.cov, A.T)) + xi.cov
    pred_cov = np.outer(phi(Xpts[0] - pred_mean), phi(Xpts[0] - pred_mean)) * W0C + xi.cov
    for ii in range(1, Xpts.shape[0]):
        pred_cov += np.outer(phi(Xpts[ii] - pred_mean), phi(Xpts[ii] - pred_mean)) * Wic

    return Gaussian(pred_mean, pred_cov)


def ukf_update(data: np.ndarray, h: Callable[[np.ndarray], np.ndarray], X: Gaussian, eta: Gaussian) -> Gaussian:
    """Linear Gaussian Inverse Problem.

    Inputs
    ------
    data: (N) array of data
    h: observation function
    X: Gaussian
    eta: Gaussian

    Returns
    -------
    Gaussian Posterior
    """
    Xpts, (W0m, Wim, W0C, Wic) = unscented_points(X)
    # mu = h(X.mean) + eta.mean
    mu = h(Xpts[0]) * W0m
    for ii in range(1, Xpts.shape[0]):
        mu += h(Xpts[ii]) * Wim

    # cant use np.dot because shapes of x and data are different
    # U = np.dot(X.cov, H.T)
    U = np.outer((Xpts[0] - X.mean) , (h(Xpts[0]) - mu)) * W0C
    for ii in range(1, Xpts.shape[0]):
        U += np.outer((Xpts[ii] - X.mean), (h(Xpts[ii]) - mu)) * Wic

    # S = np.dot(H, U) + eta.cov
    S = np.dot(h(Xpts[0] - mu), h(Xpts[0] - mu)) * W0C + eta.cov
    for ii in range(1, Xpts.shape[0]):
        S += np.dot(h(Xpts[ii] - mu) , h(Xpts[ii] - mu)) * Wic

    update_mean = X.mean + np.squeeze(U * 1/S * data - mu)
    update_cov = X.cov - np.dot(U, 1/S * U.T)
    return Gaussian(update_mean, update_cov)


def unscented_kalman_filter(data: Observations,
                            phi: Callable[[np.ndarray], np.ndarray],
                            h: Callable[[np.ndarray], np.ndarray],
                            prior: Gaussian,
                            xi: Gaussian, eta: Gaussian) -> Tuple[np.ndarray, np.ndarray]:
    """The Kalman filter.

    Inputs
    ------
    data: (N, m), N is the number of time steps, m is the size of the observations
    phi: dynamics function
    A: linearized dynamics function
    h: observation function
    H: linearized observation function
    prior: prior Gaussian
    xi: process noise
    eta: measurement noise

    Return
    ------
    KFTracker
    """

    num_steps = data.times.shape[0]

    d = prior.mean.shape[0]
    mean_store = np.zeros((num_steps, d))
    mean_store[0, :] = np.copy(prior.mean)
    cov_store = np.zeros((num_steps, d, d))
    cov_store[0, :, :] = np.copy(prior.cov)

    std_store = np.zeros((num_steps, d))
    std_store[0, :] = np.sqrt(np.diag(cov_store[0, :, :]))

    # Loop over all time steps
    Xnext = prior
    on_obs = 0
    for ii in range(1, num_steps):
        # Prediction
        Xpred = ukf_predict(phi, Xnext, xi)

        # We have an observation so an update must occur
        if on_obs < data.obs_ind.shape[0] and ii == data.obs_ind[on_obs]:
            y = data.obs[on_obs]
            on_obs += 1

            # Update
            Xup = ukf_update(y, h, Xpred, eta)
            Xnext = Xup

        else:
            Xnext = Xpred

        mean_store[ii, :] = np.copy(Xnext.mean)
        cov_store[ii, :, :] = np.copy(Xnext.cov)
        std_store[ii, :] = np.sqrt(np.diag(cov_store[ii, :, :]))

    return KFTracker(mean_store, cov_store, std_store)

ukf_results = []
ukf_start = time.time()
for obs in data:
    ukf_results.append(unscented_kalman_filter(obs.obs, xkp1, yk, prior, process_cov, Gaussian(0, obs.R)))
print(f'UKF took {time.time() - ukf_start:.3f} seconds')

for result, obs in zip(ukf_results, data):
    mse = mean_sq_error(truth, result)
    print(f'delta={obs.delta} \tR={obs.R}\tMSE={mse:.1e}')

fignum = plot_data_and_truth(fignum, 'Pendulum trajectories with UKF', truth, data, ukf_results)